In [13]:
import re
import zipfile
import pandas as pd

from pathlib import Path

Set up the output directory

In [8]:
OUTPUT_DIR = Path('../../data/processed/dfe/')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

Create a function to find the first matching filename in a zip file

In [ ]:
def get_filename(zip, pattern):
    return next(d for d in zip.namelist() if re.search(pattern, d))

Create a function to read the data

In [9]:
def get_education_dataset(csv, query='True', drop_columns=[], id_vars=[], *args, **kwargs):
    data = (
        pd.read_csv(csv, *args, **kwargs)
            .query(query)
            # Remove columns that we don't want to include
            .drop(columns=drop_columns)
            .melt(id_vars=id_vars)
        )
    data.set_index(
        data.columns[
            ~data.columns.isin(['value'])
        ].tolist(),
        inplace=True
    )
    data.value = pd.to_numeric(data.value, errors='coerce')
    
    return data

Create a function to report on the column values

In [10]:
def get_metadata(data):
    return pd.DataFrame(
        data.index.levels,
        index=data.index.names
    ).apply(
        lambda r: [x for x in r.tolist() if x is not None],
        axis=1
    )

Load KS4 data

In [6]:
with zipfile.ZipFile('../../data/raw/education/key-stage-4-performance.zip') as zip:  
    with zip.open(get_filename(zip, r'\d{4}_la_data_[a-z]+.csv$')) as csv:
        ks4 = get_education_dataset(csv,
                                    query='geographic_level == "Local authority"',
                                    dtype={
                                        'education_investment_area_flag': str,
                                        'priority_area_flag': str
                                    },
                                    id_vars=[
                                        'new_la_code',
                                        'time_period',
                                        'time_identifier',
                                        'geographic_level', 'version', 'education_investment_area_flag',
                                        'priority_area_flag', 'establishment_type', 'gender',
                                    ],
                                    drop_columns=[
                                        # Dimensions
                                        'country_code', 'la_name', 'country_name', 'region_code', 'region_name', 'old_la_code',
                                        # Measures
                                        't_schools', 't_pupils',
                                        't_att8',  # 'avg_att8',
                                        't_entbasics', 'pt_entbasics',
                                        't_l2basics_95', 'pt_l2basics_95',
                                        't_l2basics_94', 'pt_l2basics_94',
                                        't_ebacc_e_ptq_ee', 'pt_ebacc_e_ptq_ee',
                                        't_ebacc_95', 'pt_ebacc_95',
                                        't_ebacc_94', 'pt_ebacc_94',
                                        't_ebaccaps', 'avg_ebaccaps',
                                        't_inp8calc',
                                        't_p8score',
                                        # 'avg_p8score',
                                        'p8score_ci_low', 'p8score_ci_upp',
                                        't_p8eng', 'avg_p8eng', 'p8eng_ci_low', 'p8eng_ci_upp',
                                        't_p8mat', 'avg_p8mat', 'p8mat_ci_low', 'p8mat_ci_upp',
                                        't_p8ebac', 'avg_p8ebac', 'p8ebac_ci_low', 'p8ebac_ci_upp',
                                        't_p8open', 'avg_p8open', 'p8open_ci_low', 'p8open_ci_upp',
                                        't_entry_1', 'pt_entry_1',
                                        't_anypass', 'pt_anypass',
                                        't_ebaceng_e', 'pt_ebaceng_e',
                                        't_ebacmat_e', 'pt_ebacmat_e',
                                        't_ebac2sci_e', 'pt_ebac2sci_e',
                                        't_ebachum_e', 'pt_ebachum_e',
                                        't_ebaclan_e', 'pt_ebaclan_e',
                                        't_ebaceng_95', 'pt_ebaceng_95',
                                        't_ebacmat_95', 'pt_ebacmat_95',
                                        't_ebac2sci_95', 'pt_ebac2sci_95',
                                        't_ebachum_95', 'pt_ebachum_95',
                                        't_ebaclan_95', 'pt_ebaclan_95',
                                        't_ebaceng_94', 'pt_ebaceng_94',
                                        't_ebacmat_94', 'pt_ebacmat_94',
                                        't_ebac2sci_94', 'pt_ebac2sci_94',
                                        't_ebachum_94', 'pt_ebachum_94',
                                        't_ebaclan_94', 'pt_ebaclan_94',
                                        't_ebptseng_ll', 'avg_ebptseng_ll',
                                        't_ebptsmat', 'avg_ebptsmat',
                                        't_ebptssci', 'avg_ebptssci',
                                        't_ebptshum', 'avg_ebptshum',
                                        't_ebptslan', 'avg_ebptslan',
                                        't_att8_eng', 'avg_att8_eng',
                                        't_att8_mat', 'avg_att8_mat',
                                        't_att8_ebac', 'avg_att8_ebac',
                                        't_att8_open', 'avg_att8_open',
                                        't_att8_open_g', 'avg_att8_open_g',
                                        't_att8_open_ng', 'avg_att8_open_ng',
                                        't_ebac_fill', 'avg_ebac_fill',
                                        't_open_fill', 'avg_open_fill'
                                    ])

    with zip.open(get_filename(zip, r'\d{4}_la_char_data_[a-z]+.csv$')) as csv:
        ks4_char = get_education_dataset(csv,
                                         query='geographic_level == "Local authority"',
                                         dtype={
                                             'education_investment_area_flag': str,
                                             'priority_area_flag': str
                                         },
                                         id_vars=[
                                             'new_la_code',
                                             'time_period',
                                             'time_identifier',
                                             'geographic_level', 'version', 'education_investment_area_flag',
                                             'priority_area_flag', 'establishment_type', 'gender', 'breakdown',
                                             'ethnicity_major', 'free_school_meals', 'sen_status', 'sen_description', 'disadvantage', 'first_language'
                                         ],
                                         drop_columns=[
                                             # Dimensions
                                             'country_code', 'la_name', 'country_name', 'region_code', 'region_name', 'old_la_code',
                                             # Measures
                                             't_schools', 't_pupils',
                                             't_att8',
                                             # 'avg_att8',
                                             't_entbasics', 'pt_entbasics',
                                             't_l2basics_95', 'pt_l2basics_95',
                                             't_l2basics_94', 'pt_l2basics_94',
                                             't_ebacc_e_ptq_ee', 'pt_ebacc_e_ptq_ee',
                                             't_ebacc_95', 'pt_ebacc_95',
                                             't_ebacc_94', 'pt_ebacc_94',
                                             't_ebaccaps', 'avg_ebaccaps',
                                             't_inp8calc', 't_p8score',
                                             # 'avg_p8score',
                                             'p8score_ci_low', 'p8score_ci_upp'
                                         ])

KeyError: "There is no item named 'data/2223_la_data_provisional.csv' in the archive"

Interested in destinations at a Local Authority District (i.e. Lower-Tier Local Authority).

Using the KS4 Provider location as a proxy for residence LAD, as people tend to live closer to school than closer to college.

Also assuming that cohort level group is not a relevant breakdown for analysis.

In [ ]:
with zipfile.ZipFile('../../data/raw/education/16-18-destination-measures.zip') as zip:
    with zip.open(get_filename(zip, r'data/1618_dm_ud_\d{6}_la_[a-z]+.csv')) as csv:
        destinations = get_education_dataset(csv,
                                             query='geographic_level == "Local authority district" & level_methodology == "KS4 Provider location" & cohort_level_group == "Total"',
                                             id_vars=[
                                                 'time_period',
                                                 'lad_code',
                                                 'breakdown_topic',
                                                 'breakdown',
                                                 'data_type',
                                                 'version',
                                             ],
                                             drop_columns=[
                                                 # Dimensions
                                                 'country_code', 'country_name',
                                                 'region_code', 'region_name',
                                                 'old_la_code', 'new_la_code', 'la_name',
                                                 'pcon_code', 'pcon_name',
                                                 'lad_name',
                                                 'opportunity_area_code', 'opportunity_area_name',
                                                 'time_identifier',
                                                 'geographic_level',
                                                 'cohort_level_group',
                                                 'cohort_level',
                                                 'institution_group',
                                                 'institution_type',
                                                 'level_methodology',

                                                 # Variables
                                                 'all_unknown',
                                                 'all_work',
                                                 'appren',
                                                 'cohort',
                                                 'education',
                                                 'fel1',
                                                 'fel2',
                                                 'fel3',
                                                 'other_edu',

                                                 # Keep the variables below...
                                                 # 'fe', 'he', 'all_notsust', 'overall' 'appl2', 'appl3', 'appl4',
                                             ])

Save files and metadata

In [ ]:
ks4.to_csv(OUTPUT_DIR.joinpath('ks4_outcomes.csv'), index=True)
ks4.to_parquet(OUTPUT_DIR.joinpath('ks4_outcomes.parquet'), index=True)
ks4.pipe(get_metadata).to_json(OUTPUT_DIR.joinpath('ks4_outcomes.metadata.json'), indent=2)

ks4_char.to_csv(OUTPUT_DIR.joinpath('ks4_char_outcomes.csv'), index=True)
ks4_char.to_parquet(OUTPUT_DIR.joinpath('ks4_char_outcomes.parquet'), index=True)
ks4_char.pipe(get_metadata).to_json(OUTPUT_DIR.joinpath('ks4_char_outcomes.metadata.json'), indent=2)

destinations.to_csv(OUTPUT_DIR.joinpath('destinations.csv'), index=True)
destinations.to_parquet(OUTPUT_DIR.joinpath('destinations.parquet'), index=True)
destinations.pipe(get_metadata).to_json(OUTPUT_DIR.joinpath('destinations.metadata.json'), indent=2)

Example query of characteristic data: filter by `breakdown` to start with (and then any other queries). The associated column will then have the breakdown, and all other breakdowns will be set to Total. Refer to the metadata.json files for information about the values that can be queried.

In [ ]:
ks4_char.reset_index().query("breakdown == 'Free school meals' and time_period == 202223")